# Google Play

**Install library jika belum di-install**

In [1]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.0 MB/s eta 0:00:00


Opsi yang terdapat pada fungsi reviews: source : https://pypi.org/project/google-play-scraper/

1. ID aplikasi yang ada pada Google Play,
didapat dari link aplikasi pada Google Play setelah sama dengan(=). Contoh https://play.google.com/store/apps/details?id=com.ea.game.pvzfree_row maka ID aplikasinya adalah com.ea.game.pvzfree_row
2. lang: Bahasa yang digunakan dalam ulasan. Reverensi singkatan nama negara
3. country: Asal negara ulasan. Reverensi singkatan nama negara
4. sort: Mode pengurutan ulasan. Bisa menggunakan Sort.MOST_RELEVANT (Paling relevan) atau Sort.NEWEST (Paling baru).
5. count: Jumlah ulasan yang ingin diambil. Jika menggunakan script reviews_all, maka akan mengambil semua ulasan yang ada dan mengabaikan jumlah count.
6. filter_score_with: Rating ulasan (1 sd 5), artinya apabila diisi 1 maka ulasan yang diambil hanya ulasan pada rating 1, atau bisa diisi None untuk mengambil ulasan pada semua rating.

In [2]:
from google_play_scraper import app, reviews, reviews_all, Sort

In [ ]:
detail_app = app(
    'com.ea.game.pvzfree_row',
    lang='id',
    country='id'
)
detail_app

In [4]:
scrap_app, continuation_token = reviews(
    'com.ea.game.pvzfree_row',
    lang='en',
    country='uk',
    sort=Sort.NEWEST,
    count = 10000,
    filter_score_with = None
)

**Menyimpan Ulasan dalam file CSV atau JSON**

Sebagaimana diketahui Pandas merupakan paket python yang memberikan kemudahan dalam memproses dan menganalisa data. Biasanya data-data yang telah diolah disimpan dalam file, dan yang paling umum adalah file dengan format csv atau (comma separated value).

Untuk melakukan export Pandas DataFrame ke file csv, Pandas memiliki berbagai cara, diantaranya dengan memanfaatkan library csv atau menggunakan fungsi .to_csv.

source:

https://hub.idbigdata.com/sigit-prasetyo/export-pandas-dataframe-sebagai-file-csv--47

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [5]:
# Menyimpan data dengan library csv

import csv
with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrap_app:
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

In [6]:
# Menyimpan data dengan fungsi .to_csv
import pandas as pd

scrap_df = pd.DataFrame(scrap_app)
scrap_df.to_csv('ulasan_apk.csv',# Nama file yang akan disimpan (diakhiri dengan .csv)
                index = False,   # Apakah index data akan ikut dimasukan kedalam file csv?
                sep = ',',       # Tanda yang memisahkan antar kolom (default: ,)
                header = True,   # Apakah header atau nama kolom disertakan pada file csv?
                mode = 'w'       # Mode pembuatan file memotong jika sudah ada
                )

In [7]:
# Menyimpan data dengan fungsi .to_json

scrap_df.to_json('ulasan_apk.json')

# Youtube

Link untuk mendapatkan Youtube API Key: https://console.cloud.google.com/

Link dokumentasi Google Youtube Data API:
https://developers.google.com/youtube/v3

Link untuk membaca JSON: https://jsonformatter.curiousconcept.com/

In [8]:
import pandas as pd
import requests
import json
import googleapiclient.discovery
from googleapiclient.discovery import build

## Cara 1

In [31]:
api_key = 'IzSyBo-6OStkEO5rOwSOToRk'
video_id = 'XefDBZJvfJE'
youtube = build('youtube', 'v3', developerKey=api_key)
video_response = youtube.commentThreads().list(part='snippet, replies', videoId=video_id).execute()

In [ ]:
video_response

In [36]:
replies = []

for item in video_response['items']:
  published = item['snippet']['topLevelComment']['snippet']['publishedAt']
  user = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
  comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
  likecount = item['snippet']['topLevelComment']['snippet']['likeCount']
  replies.append([published, user, comment, likecount])

  replycount = item['snippet']['totalReplyCount']
  if replycount > 0:
    for reply in item['replies']['comments']:
      published = reply['snippet']['publishedAt']
      user = reply['snippet']['authorDisplayName']
      repl = reply['snippet']['textDisplay']
      likecount = reply['snippet']['likeCount']
      replies.append([published, user, repl, likecount])

  if 'nextPageToken' in video_response:
    video_response = youtube.commentThreads().list(
    part='snippet, replies',
    videoId=video_id,
    pageToken=video_response['nextPageToken']).execute()
  else:
    break

In [ ]:
side_1 = pd.DataFrame(replies)
side_1.columns = ['published_date', 'user', 'comment', 'like']
side_1

In [ ]:
print('Jumlah data: ', len(side_1))
print('Data terduplikasi: ', side_1.duplicated().sum())
print('Data kosong: ', side_1.isnull().sum())

## Cara 2

In [25]:
info_url = 'https://www.googleapis.com/youtube/v3/videos?part=snippet&id=C_fOUCHb5L0&key=IzSyBo-6OStkEO5rOwSOToRk'
comment_url = 'https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=C_fOUCHb5L0&key=IzSyBo-6OStkEO5rOwSOToRk'

In [ ]:
response = requests.get(info_url)
response.json()

In [ ]:
comment = requests.get(comment_url)
comment.json()

In [ ]:
comments = [item['snippet']['topLevelComment']['snippet']['textDisplay'] for item in comment.json()['items']]
likes = [item['snippet']['topLevelComment']['snippet']['likeCount'] for item in comment.json()['items']]
side_2 = list(zip(comments, likes))
side_2

In [ ]:
side_2 = pd.DataFrame(side_2)
side_2.head()

In [ ]:
print('Jumlah data: ', len(side_2))
print('Data terduplikasi: ', side_2.duplicated().sum())
print('Data kosong: ', side_2.isnull().sum())

## Cara 3

In [40]:
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY
)

In [44]:
num = 0
result = []
page_token = None
max = 10000

while num < max:
  request = youtube.commentThreads().list(
  part = 'snippet, replies',
  videoId = 'dhYOPzcsbGM',
  pageToken = page_token,
  maxResults = 100
  )
  response = request.execute()

  for i in range (0, len(response['items'])):
    published_date = response['items'][i]['snippet']['topLevelComment']['snippet']['publishedAt']
    user = response['items'][i]['snippet']['topLevelComment']['snippet']['authorDisplayName']
    comment = response['items'][i]['snippet']['topLevelComment']['snippet']['textDisplay']
    like = response['items'][i]['snippet']['topLevelComment']['snippet']['likeCount']
    result.append([published_date, user, comment, like])
    num += 1

  page_token = response.get('nextPageToken')

In [ ]:
side_3 = pd.DataFrame(result)
side_3.columns = ['published_date', 'user', 'comment', 'like']
side_3.head()

In [ ]:
print('Jumlah data: ', len(side_3))
print('Data terduplikasi: ', side_3.duplicated().sum())
print('Data kosong: ', side_3.isnull().sum())

In [47]:
with open('ulasan_youtube.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])
    for review in scrap_app:
        writer.writerow([review['content']])